# Cleaning TVG

In this notebook, I clean the TVG data and put it in a dataframe

In [2]:
import pandas as pd
import glob
import os

In [160]:
data = '../../../datasets/tvg_data/'
allFiles = glob.glob(data + "/**/*.txt", recursive=True)

In [165]:
corpus = []
volume = []
name = []
documents = []
author_ = []
title_ = []
for fname in os.listdir(data):
    volume = fname.split()
    path = os.path.join(data, fname)
    if os.path.isdir(path):
        allFiles = sorted(glob.glob(path + "/*.txt", recursive=True))
        content = glob.glob(path + "/*.csv")
        content_df = pd.read_csv(content[0], delimiter='\t')
        for file_ in allFiles:
            author = []
            page = os.path.basename(file_)
            page = page.split("page_")[1][:-4]
            try:
                author = content_df['author'][(content_df['page_first'] <= int(page)) & (content_df['page_last'] > int(page))].max()
                title = content_df['title'][(content_df['page_first'] <= int(page)) & (content_df['page_last'] > int(page))].max()
            except:
                pass
            with open(file_, 'r') as f:
                author_.append(author)
                title_.append(title)
                volume.append(fname)
                name.append(page)
                documents.append(f.read().replace('\n', ' '))

In [166]:
df = pd.DataFrame(list(zip(volume, name, author_, title_, documents)),
              columns=['volume','page', 'author', 'title', 'documents'])

In [184]:
def get_edition_by_year(year: int):
    missing_editions = [1888, 1944, 1945]
   # 1888 exists but is not retro-digitized
   # 1944 and 1945 were skipped
    if year < 1886 or year > 2008:
        return None
    if year in missing_editions: # no editions
        return None
    if year < 1944:
        edition = str(year - 1885)
    elif year > 1945:
        edition = str(year - 1887)
    if year == 2008:
        # edition 121 and 122 are both 2008, 122 is copy of 121 but better digitization
        edition = "122"
    if len(edition) == 1:
        edition = "0" + edition
    return "tvg_" + edition

    edition_of_year = {}
    year_of_edition = {}


In [192]:
for year in range(1885, 2010):
    edition = get_edition_by_year(year)
    if edition:
        edition_of_year[year] = edition
        year_of_edition[edition] = str(year)

In [193]:
df["year"] = df['volume'].map(year_of_edition)

In [195]:
df.to_csv('tvg.csv')

In [4]:
df = pd.read_csv('../data/TVG/tvg.csv')